In [10]:
from __future__ import print_function
import pandas as pd
from pymongo import MongoClient
import collections
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

In [17]:
def retrieve_from_prod_db():
    '''
    helper function to munge and explore with pandas
    '''
    client = MongoClient()
    db = client.prod
    cursor = db.messages.find({})
    return pd.DataFrame(list(cursor))


def messages_to_words(df):
    '''
    '''
    # remove Nan stuff and not incoming
    df = df[(df.incoming == True) & (df.msg.isnull() == False)]
    action_codes_dictionary = {}
    df['action_codes'] = df['action'].astype('category')
    action_codes = np_utils.to_categorical(df['action_codes'].cat.codes)
    i = 0
    for k in df.action_codes.cat.categories:
        action_codes_dictionary[k] = i
        i += 1
    action_codes = np_utils.to_categorical(df.action_codes.values)

    df.loc[:, 'msg'] = df.loc[:, 'msg'].str.lower()
    words = df.msg.str.split(' ')
    word_list = [w for sublist in words.values for w in sublist]
    return word_list, words, action_codes


def build_dataset(words, vocabulary_size=50000):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary


def words2numbers(words, dictionary):
    '''
    '''
    data = []
    for row in words:
        data.append([dictionary[w] for w in row])
    return data

In [13]:
def data_for_model():
    df = retrieve_from_prod_db()
    # df = dict_to_cols(df)
    words, word_list, targets = messages_to_words(df)
    data, count, dictionary, reverse_dictionary = build_dataset(words)
    data = words2numbers(word_list.values, dictionary)
    data = pad_sequences(data, maxlen=20)
    return data, targets, dictionary, reverse_dictionary

In [97]:
df = retrieve_from_prod_db()

In [98]:
df = df[(df.incoming == True) & (df.msg.isnull() == False)]

In [100]:
df['action_codes'] = df['action'].astype('category')

In [103]:
action_codes = np_utils.to_categorical(df.action_codes.cat.codes.values)

In [104]:
action_codes_dictionary = {}
i = 0
for k in df.action_codes.cat.categories:
    action_codes_dictionary[k] = i
    i += 1

rev_action_codes_dictionary = dict((v, k) for k, v in action_codes_dictionary.iteritems())

In [105]:
df.loc[:, 'msg'] = df.loc[:, 'msg'].str.lower()
words = df.msg.str.split(' ')
word_list = [w for sublist in words.values for w in sublist]

In [106]:
data, count, dictionary, reverse_dictionary = build_dataset(word_list)
data = words2numbers(words.values, dictionary)
data = pad_sequences(data, maxlen=20)

In [109]:
print('data shape: ',data.shape)
print('target shape: ',targets.shape)

data shape:  (18078, 20)
target shape:  (18078, 16)


In [110]:
data[5]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        2, 18, 12], dtype=int32)

In [111]:
[dictionary[x] for x in df.msg.iloc[5].split(' ')]

[2, 18, 12]

In [57]:
df.iloc[5]

__v                                                             0
_id                                      5683f9049b53498b79422d11
action                                                    similar
amazon                                                         []
bucket                                                     search
click                                                         NaN
client_res                                                     []
dataModify                                           {u'val': []}
flags                                                         NaN
incoming                                                     True
mode                                                          NaN
msg                                                   more like 2
resolved                                                    False
slackData                                                     NaN
source          {u'origin': u'slack', u'org': u'T0HGFKHKJ', u'...
thread    

In [68]:
len(count)

5204

In [130]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Dropout, LSTM, merge

In [71]:
action_codes[5]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.])

In [121]:
rev_action_codes_dictionary[action_codes[5].argmax()]

u'similar'

In [126]:
data.shape[1]

20

In [124]:
len(dictionary)

5204

In [131]:
sequence = Input(shape=(data.shape[1],), dtype='int32')
embedded = Embedding(len(dictionary), 128, input_length=data.shape[1])(sequence)

# apply forwards + backwards LSTM
forwards = LSTM(64)(embedded)
backwards = LSTM(64, go_backwards=True)(embedded)

# concatenate the outputs of the 2 LSTMs
merged1 = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged1)
output = Dense(action_codes.shape[1], activation='softmax')(after_dp)

model = Model(input=sequence, output=output)

# try using different optimizers and different optimizer configs
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(data, action_codes, validation_split=.2, nb_epoch=1, batch_size=32)

WARNING (theano.sandbox.cuda.opt): Optimization Warning: Got the following error, but you can ignore it. This could cause less GpuElemwise fused together.
(4, -65538)
ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: GpuFromHost(TensorConstant{(1, 1) of 1e-07})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/graham/.virtualenvs/cinna-slack/lib/python2.7/site-packages/theano/gof/opt.py", line 1772, in process_node
    replacements = lopt.transform(node)
  File "/Users/graham/.virtualenvs/cinna-slack/lib/python2.7/site-packages/theano/tensor/opt.py", line 5827, in constant_folding
    required = thunk()
  File "/Users/graham/.virtualenvs/cinna-slack/lib/python2.7/site-packages/theano/gof/op.py", line 883, in rval
    fill_storage()
  File "/Users/graham/.virtualenvs/cinna-slack/lib/python2.7/site-packages/theano/gof/cc.py", line 1707, in __call__
    reraise(exc_type, exc_value,